# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [43]:
# import numpy and pandas
import numpy as np
import pandas as pd
from scipy import stats
import math

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [12]:
# Your code here:

legendary = pokemon[pokemon['Legendary']]
legendary.shape[0]

65

In [13]:
not_legendary = pokemon[pokemon['Legendary'] == False]
not_legendary.shape[0]

735

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [44]:
# Your code here:
mean_legendary = np.mean(legendary['Total'])
std_legendary = np.std(legendary['Total'])

print(mean_legendary,std_legendary)

mean_not_legendary = np.mean(not_legendary['Total'])
std_not_legendary = np.std(not_legendary['Total'])

print(mean_not_legendary, std_not_legendary)

637.3846153846154 60.46682302213781
417.21360544217686 106.68776660321996


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [59]:
# Your code here:

stats.ttest_ind(legendary['Total'], not_legendary['Total'], equal_var=False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

In [122]:
mean_diference = np.mean(legendary['Total']) - np.mean(not_legendary['Total'])

n_legendary = legendary['Total'].shape[0]
n_not_legendary = not_legendary['Total'].shape[0]
std_legendary = np.std(legendary['Total'])
std_not_legendary = np.std(not_legendary['Total'])
v1 = np.var(legendary['Total'])
v2 = np.var(not_legendary['Total'])


t = mean_diference / math.sqrt(((std_legendary ** 2)/n_legendary) + ((std_not_legendary ** 2)/n_not_legendary))

v = ((v1/n_legendary + v2/n_not_legendary) ** 2)/(((v1/n_legendary)**2)/(n_legendary-1)+((v2/n_not_legendary)**2)/(n_not_legendary-1))

pvalue = stats.t(v).cdf(-t)

print(t,pvalue)

25.995123783608943 1.8841110117222902e-47


What do you conclude from this test? Write your conclusions below.

In [46]:
# Your conclusions here:



25.995123783608943

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [25]:
# Your code here:

pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Ground       32
Ghost        32
Dragon       32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [30]:
# Your code here:
water = pokemon[pokemon['Type 1'] == 'Water']
mean_water = np.mean(water['Total'])
std_water = np.std(water['Total'])

not_water = pokemon[pokemon['Type 1'] != 'Water']
mean_not_water = np.mean(not_water['Total'])
std_not_water = np.std(not_water['Total'])



Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [96]:
# Your code here:
stats.ttest_ind(water['Total'], not_water['Total'], equal_var=True)


Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

In [124]:
d = mean_water - mean_not_water

v1 = np.var(water['Total'])
v2 = np.var(not_water['Total'])
n1 = water['Total'].shape[0]
n2 = not_water['Total'].shape[0]

sp = math.sqrt(((n1 - 1) * v1 +(n2 - 1)* v2) / (n1 + n2 - 2))
t = d/(sp*(math.sqrt(1/n1 + 1/n2)))
pvalue = stats.t(n1 + n2 - 2).cdf(t)

print(t,pvalue*2)

-0.4423810933856032 0.6583333307531565


## Write your conclusion below.

In [10]:
# Your conclusions here:



# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [125]:
# Your code here:
attack = pokemon['Attack']
defense = pokemon['Defense']

stats.ttest_rel(attack,defense)

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

In [134]:
n = attack.shape[0]
diference = attack - defense
mean_diference = np.mean(diference)
std_diference = np.std(diference)
st_diference_n = std_diference / math.sqrt(n)


t = mean_diference/st_diference_n
pval = stats.t(n-1).cdf(-t)

print(t, pval*2)

4.3282724094808795 1.6935882019689846e-05


Describe the results of the test in the cell below.

In [135]:
# Your conclusions here:
pokemon.columns


Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [137]:
# Your code here:
attack = pokemon['Sp. Atk']
defense = pokemon['Sp. Def']

stats.ttest_rel(attack,defense)


Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

In [138]:
n = attack.shape[0]
diference = attack - defense
mean_diference = np.mean(diference)
std_diference = np.std(diference)
st_diference_n = std_diference / math.sqrt(n)


t = mean_diference/st_diference_n
pval = stats.t(n-1).cdf(-t)

print(t, pval*2)

0.8545204307254721 0.3930728301195733


Describe the results of the test in the cell below.

In [143]:
# Your conclusions here:
stats.ttest_1samp(diference, 0)


Ttest_1sampResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [16]:
# Your code here:
    
    

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [148]:
# Your code here:
df = pd.crosstab(pokemon['Legendary'],pokemon['Type 1'] == 'Water')
df

Type 1,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [155]:
# Your code here:
chi2, p, dof, ex = stats.chi2_contingency(df,correction=False)

print(chi2, p)

3.617455159932616 0.0571762700873503


In [162]:

def nCr(n,r):
    f = math.factorial
    return f(n) / (f(r) * f(n-r))



In [165]:
a = df.iloc[0,0]
b = df.iloc[0,1]
c = df.iloc[1,0]
d = df.iloc[1,1]
n =  a + b + c + d

p = (nCr(a + b,a) * nCr(c + d, c))/ nCr(n,a+c)



0.0464774702930065

In [166]:
stats.fisher_exact(df)

(0.3806921675774135, 0.06169310652287763)

Based on a 95% confidence, should we reject the null hypothesis?